-> clean the pickle file 

-> create a function that filters by what it needs to and add a column with the sentiment score

In [1]:
import pandas as pd
from helpermodules import memory_handling as mh
import numpy as np

In [7]:
# downloading the pickle file 
file = "fedspeechees_sentiment_2024.pkl"
helper = mh.PickleHelper.pickle_load(file)
dfnew = helper.obj

In [12]:
# downloading the pickle file 
file = "fedspeeches_preprocessed.pkl"
helper = mh.PickleHelper.pickle_load(file)
df_speech = helper.obj

In [14]:
def clean_df(df):
    """
    Clean the dataframe by sorting and filtering based on date and weekdays.

    Parameters:
    df : pandas.DataFrame
        A dataframe containing at least 'date' and 'timestamp' columns.

    Returns:
    pandas.DataFrame
        A dataframe sorted by 'date' and 'timestamp', filtered to include only
        rows with dates >= '2020-12-01' and weekdays (Monday to Friday).
    """
        
    df_ordered = df.sort_values(by=['date', 'timestamp'], ascending = [True, True])
    df_final = df_ordered[df_ordered.date >= "2024-01-01"]
    df_final = df_final[df_final['date'].dt.weekday<5]
    return df_final

In [10]:
dfnew = dfnew.sort_values(['date', 'timestamp'] , ascending=True)

In [16]:
df_speech = clean_df(df_speech)


---------

In [44]:


def add_sentiment_score(df_speech, dfnew):
    """
    Adds the 'finbert_score' from `dfnew` to `df_speech` based on matching 'speaker', 'date', and 'timestamp'.
    This method uses pandas' merge function to efficiently join the dataframes.

    Parameters:
    df_speech (pd.DataFrame): DataFrame containing speech details with columns like
                               'speaker', 'date', 'text', 'timestamp', etc.
    dfnew (pd.DataFrame): DataFrame containing sentiment scores with columns 'speaker',
                          'date', 'timestamp', and 'finbert_score'.

    Returns:
    pd.DataFrame: A modified version of `df_speech` with a new column 'finbert_score'
                  added from `dfnew` for matching rows based on 'speaker', 'date', and 'timestamp'.
    """

    # Ensure the 'date' columns are in the same datetime format
    df_speech['date'] = pd.to_datetime(df_speech['date'])
    dfnew['date'] = pd.to_datetime(dfnew['date'])

    #store the index
    df_speech_reset = df_speech.reset_index()


    # Merge df_speech with dfnew on 'speaker', 'date', and 'timestamp'
    df_merged = pd.merge(df_speech_reset, dfnew[['speaker', 'date', 'timestamp', 'finbert_score']],
                          how='left')
    
    df_merged.set_index('index', inplace=True)

    return df_speech


In [45]:
dffinal = add_sentiment_score(df_speech , dfnew )

In [46]:
dffinal

,date,speaker,title,link,text,timestamp,text_by_minute,finbert_score
24286,2024-01-08,Governor Michelle W. Bowman,New Year’s Resolutions for Bank Regulatory Pol...,/newsevents/speech/bowman20240108a.htm,It is a pleasure to join you this afternoon fo...,2024-01-08 10:00:00-05:00,It is a pleasure to join you this afternoon fo...,0.154481
24287,2024-01-08,Governor Michelle W. Bowman,New Year’s Resolutions for Bank Regulatory Pol...,/newsevents/speech/bowman20240108a.htm,It is a pleasure to join you this afternoon fo...,2024-01-08 10:01:00-05:00,offer my thoughts on the economy and monetary ...,-0.861466
24288,2024-01-08,Governor Michelle W. Bowman,New Year’s Resolutions for Bank Regulatory Pol...,/newsevents/speech/bowman20240108a.htm,It is a pleasure to join you this afternoon fo...,2024-01-08 10:02:00-05:00,"year, which may be a sign that labor market su...",-0.600611
24289,2024-01-08,Governor Michelle W. Bowman,New Year’s Resolutions for Bank Regulatory Pol...,/newsevents/speech/bowman20240108a.htm,It is a pleasure to join you this afternoon fo...,2024-01-08 10:03:00-05:00,to prevent policy from becoming overly restric...,-0.895699
24290,2024-01-08,Governor Michelle W. Bowman,New Year’s Resolutions for Bank Regulatory Pol...,/newsevents/speech/bowman20240108a.htm,It is a pleasure to join you this afternoon fo...,2024-01-08 10:04:00-05:00,"outlook, I will continue to watch the data clo...",-0.165666
...,...,...,...,...,...,...,...,...
22472,2024-10-23,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 10:02:00-04:00,"new technology. Second, we must be open in con...",0.280051
22473,2024-10-23,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 10:03:00-04:00,the future.But we must recognize that the succ...,0.056339
22474,2024-10-23,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 10:04:00-04:00,"with President Harker for the past six years, ...",0.271959
22475,2024-10-23,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-10-23 10:05:00-04:00,the application of both hard and soft data.Thi...,0.289922
